In [ ]:
import time
import urllib.request as req1
import requests as req2
from bs4 import BeautifulSoup
import pandas as pd
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 

In [ ]:
def image_save(url,i):
    save_name='data/book_image/image'+str(i)+'.jpg'
    req1.urlretrieve(url,save_name)
    print(save_name,'가 저장됨')

In [ ]:
driver=webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)
driver.get('http://www.yes24.com')
# 검색 창
target=driver.find_element(By.CSS_SELECTOR,"[name = 'query']")

# 검색할 키워드
key='python'
# 검색 창에 키워드를 검색
target.send_keys(key)
target.send_keys(Keys.ENTER) # target.submit() 동일
# 기본 주소
base="http://www.yes24.com"
# 검색 쿼리
query="/Product/Search?domain=ALL&query=%s"%key
# 이미지 변수
i=0
# 출력된 책을 담는 배열
book_list=[]
stopFlag=False
while True:
    for n in range(1,12):
        if n==1:
            link='#goodsListWrap > div.sGoodsPagen > div > strong'
        else:
            link='#goodsListWrap > div.sGoodsPagen > div > a:nth-child('+str(n)+')'
        linkNum=driver.find_element(By.CSS_SELECTOR,link) 
        linkNum.click()
        # 페이지 설정
        page="&page=%s"%n
        # url 생성
        url=base+query+page
        # url 열기
        res=req1.urlopen(url)
        # url 파싱
        bs=BeautifulSoup(res,'html.parser')
        # 책목록 부르기
        books=bs.select('#yesSchList > li')
        time.sleep(2)
        # 책목록에서 책정보들 부르기
        for book in books:
            # 제목
            title=book.select('div > div.item_info > div.info_row.info_name > a.gd_name')
            print(title[0].text)
            # 출판사
            pub=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_pub > a')
            if len(pub)==0:
                pub='출판사 없음'
                print(pub)
            else:
                pub=pub[0].text
                print(pub)
            # 작가
            author=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_auth')
            if len(author)==0:
                author='작가 없음'
                print(author)
            else:
                author=author[0].text.strip()
                print(author)
            # 출판일
            date=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_date')
            print(date[0].text)
            # 가격
            price=book.select('div > div.item_info > div.info_row.info_price > strong > em')
            print(price[0].text)
            # 책 정보를 튜플로 생성
            book_info=((title[0].text.strip(),pub,author,date[0].text,price[0].text))
            # 책 리스트에 정보 담기
            book_list.append(book_info)
            # 이미지 저장할 변수
            i+=1
            # 해당하는 책 이미지 부르기
            img=book.select('div > div.item_img > div.img_canvas > span > span > a > em > img')
            # 원본 이미지 부르기
            img_src=img[0]['data-original']
            # 이미지 url 출력
            print(img_src)
            # 이미지 저장
            image_save(img_src,i)
        # 11페이지가 끝나면 for문을 종료
        if n==11:
            stopFlag=True
            break
        print('-------------------------------------------------------')
    # 11페이지의 책 정보를 가져오지 않고 break문으로 while문을 빠져나옴
    if stopFlag==True:
        break
    nextBlock='#goodsListWrap > div.sGoodsPagen > div > a.bgYUI.next'
    nextNum=driver.find_element(By.CSS_SELECTOR,nextBlock)
    nextNum.click()
    time.sleep(2)
driver.quit()

In [ ]:
df=pd.DataFrame(book_list,columns=['title','author','pub','date','price'])
df.to_csv('data/books.csv',index=False)
df

In [ ]:
conn=pymysql.connect(host='localhost',user='jhs',password='1234',db='pydb',charset='utf8')
cursor=conn.cursor()
sql="insert into yes_book(title,author,pub,date,price) values(%s,%s,%s,%s,%s)"
cursor.executemany(sql,book_list)
conn.commit()
conn.close()